In [1]:
%matplotlib widget

In [2]:
from glob import glob
import numpy as np
import pandas as pd
import flammkuchen as fl
from scipy.interpolate import interp1d 
import json
import matplotlib.pyplot as plt

from scipy.interpolate import interp1d 
from scipy.signal import convolve2d
from fimpylab.core.lightsheet_experiment import LightsheetExperiment

from pathlib import Path

In [3]:
master = Path(r"Z:\Hagar\E0040\2p ablations\e0040v10")
fish_list = list(master.glob("*f*"))
path = fish_list[1]
print(path)

Z:\Hagar\E0040\2p ablations\e0040v10\230321_f10


In [126]:
traces = fl.load(path / "filtered_traces.h5", "/undetr")
coords = fl.load(path / "data_from_suite2p_unfiltered.h5", "/coords")
#suite2p_brain = fl.load(path / "data_from_suite2p_cells_brain.h5")
#in_brain_idx = suite2p_brain['coords_idx']

In [127]:
exp = LightsheetExperiment(path)
fs = exp.fn
stim_log = exp.stimulus_log

In [128]:
coherence = np.asarray(stim_log.random_dots_coherence)
frozen = np.asarray(stim_log.random_dots_frozen)
stim_t = np.asarray(stim_log.t)

In [129]:
fig, ax = plt.subplots(1,1)
ax.plot(stim_t, coherence)
#ax.plot(stim_t, frozen)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [130]:
fig2, ax2 = plt.subplots(1,1)
coherence_values = [-1, -0.3, 0.3, 1]
regressors = np.zeros((4, np.shape(stim_t)[0]))
for i in range(4):
    curr_coh = coherence_values[i]
    ind = np.where(coherence == curr_coh)[0]
    regressors[i, ind] = 1
    ax2.plot(regressors[i])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [131]:
sampling = 1 / fs
upsampling = 5

dt_upsampled = sampling / upsampling
t_imaging_up = np.arange(0, stim_log.t.values[-1], dt_upsampled)
reg_up = interp1d(stim_log.t.values, regressors, axis=1, fill_value="extrapolate")(
    t_imaging_up
)

# 6s kernel
u_steps = t_imaging_up.shape[0]
u_time = np.arange(u_steps) * dt_upsampled
decay = np.exp(-u_time / (1.5 / np.log(2)))
kernel = decay / np.sum(decay)

convolved = convolve2d(reg_up, kernel[None, :])[:, 0:u_steps]
reg_sensory = convolved[:, ::upsampling]

In [132]:
fig3, ax3 = plt.subplots(1,1)
for i in range(4):
    ax3.plot(reg_sensory[i])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [133]:
traces_ib = traces#[:, in_brain_idx]
coords_ib = coords#[in_brain_idx]

In [134]:
regi = reg_sensory @ traces_ib[:, :]

In [135]:
with open(next(Path(path).glob("*metadata.json")), "r") as f:
        metadata = json.load(f)
lsconfig = metadata["imaging"]["microscope_config"]['lightsheet']['scanning']
z_tot_span = lsconfig["z"]["piezo_max"] - lsconfig["z"]["piezo_min"]
n_planes = lsconfig["triggering"]["n_planes"]
z_res = z_tot_span / n_planes

In [136]:
num_traces = np.shape(traces_ib)[1]
print(num_traces)
reg_corr = np.zeros((num_traces))
for i in range(num_traces):
    reg_corr[i] = np.corrcoef(reg_sensory[0,:], traces_ib[:,i])[0,1]

38491


In [137]:
colors = reg_corr

In [138]:
fig, axs = plt.subplots(2, 4, figsize=(12, 6), gridspec_kw={'width_ratios': [5, 2, 5, 2], 'height_ratios': [1, 3]})
mp_ind = np.argsort(np.abs(colors))
axs[1,0].scatter(coords_ib[mp_ind,2]*0.6, coords_ib[mp_ind,1]*.6, c=colors[mp_ind], s=2, alpha=0.8, cmap='coolwarm', vmin=-1, vmax=1)
axs[1,1].scatter(coords_ib[mp_ind,0]*z_res, coords_ib[mp_ind,1]*0.6, c=colors[mp_ind], alpha=0.8, cmap='coolwarm', vmin=-1, vmax=1)
axs[0,0].scatter(coords_ib[mp_ind,2]*0.6, coords_ib[mp_ind,0]*z_res, c=colors[mp_ind], alpha=0.8, cmap='coolwarm', vmin=-1, vmax=1)

axs[0,0].spines['right'].set_visible(False)
axs[0,0].spines['top'].set_visible(False)

axs[1,1].spines['right'].set_visible(False)
axs[1,1].spines['top'].set_visible(False)

axs[1,0].spines['right'].set_visible(False)
axs[1,0].spines['top'].set_visible(False)

axs[0,1].axis('off')
axs[0,3].axis('off')

axs[1,2].scatter(coords_ib[:,2]*0.6, coords_ib[:,1]*.6, c=colors[:], s=2, alpha=0.8, cmap='coolwarm', vmin=-1, vmax=1)
axs[1,3].scatter(coords_ib[:,0]*z_res, coords_ib[:,1]*0.6, c=colors[:], alpha=0.8, cmap='coolwarm', vmin=-1, vmax=1)
axs[0,2].scatter(coords_ib[:,2]*0.6, coords_ib[:,0]*z_res, c=colors[:], alpha=0.8, cmap='coolwarm', vmin=-1, vmax=1)

axs[0,2].spines['right'].set_visible(False)
axs[0,2].spines['top'].set_visible(False)

axs[1,3].spines['right'].set_visible(False)
axs[1,3].spines['top'].set_visible(False)

axs[1,2].spines['right'].set_visible(False)
axs[1,2].spines['top'].set_visible(False)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [139]:
fig.savefig(path / "negative_coherence 1.jpg", dpi=300)